---
## Python Packages & Directories
---

In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from __future__ import (division, print_function, absolute_import,
                        unicode_literals)
import  sys, os, glob, time, IPython

import astropy.constants as const
import astropy.units as u

from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
# from astropy.utils.data import get_pkg_data_filename
from astropy.coordinates import SkyCoord, EarthLocation

# from PyAstronomy import pyasl

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import AutoMinorLocator
plt.rcParams["font.family"] = "Times New Roman"

import numpy as np
import scipy as sp
import pandas as pd
import re

import seaborn as sns
sns.set_palette("colorblind")
colors = sns.color_palette("colorblind", 10) + sns.color_palette("Paired", 10)[2:] + sns.color_palette("Set2", 10)

# from smh import Session

import spag.read_data as rd
import spag.utils as spagu
from spag.convert import *
import spag.coordinates as coord

script_dir = "/".join(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"].split("/")[:-1]) + "/"
# script_dir = os.path.dirname(os.path.realpath(__file__))+"/"


        Use @ instead. [gala.coordinates.sgr]
        Use @ instead. [gala.coordinates.orphan]
        Use @ instead. [gala.coordinates.magellanic_stream]


In [3]:
## Show all columns and rows of the dataframe
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows

---
## Loading and Modifying Datafile
---

In [4]:
sestito2024_df = pd.read_csv("/Users/ayelland/Research/metal-poor-stars/spag/spag/data/abundance_tables/sestito2024/pigs_ix_tab4-ay.csv")

## Remove columns and round the data values
for col in sestito2024_df.columns:
    if col.startswith("o_[") or col.startswith("s_["):
        sestito2024_df.drop(col, axis=1, inplace=True)
    
    if col.startswith("[") or (col in ['Teff','e_Teff','logg','e_logg']) or col.startswith("e_"):
        ## Round the values to 2 decimal places
        sestito2024_df[col] = sestito2024_df[col].apply(lambda x: normal_round(x, 2))

sestito2024_df.rename(columns={'GaiaDR3': 'Simbad_Identifier'}, inplace=True)
sestito2024_df['Simbad_Identifier'] = 'Gaia DR3 ' + sestito2024_df['Simbad_Identifier'].astype(str)
sestito2024_df['Loc'] = 'DW'
sestito2024_df['Reference'] = 'Sestito+2024'
sestito2024_df['Ref'] = 'SES24'
sestito2024_df.drop(columns=['[Fe/H]F'], inplace=True)

## Carbon Correction calculation
sestito2024_df['[C/Fe]f'] = (sestito2024_df['[C/Fe]'] + sestito2024_df['[C/Fe]c']).apply(lambda x: normal_round(x, 2))

## Fill in the RA and DEC columns
if 'RA_hms' not in sestito2024_df.columns:
    sestito2024_df['RA_hms'] = np.nan
if 'DEC_dms' not in sestito2024_df.columns:
    sestito2024_df['DEC_dms'] = np.nan
if 'RA_deg' not in sestito2024_df.columns:
    sestito2024_df['RA_deg'] = np.nan
if 'DEC_deg' not in sestito2024_df.columns:
    sestito2024_df['DEC_deg'] = np.nan

for idx, row in sestito2024_df.iterrows():
    if pd.isna(row['RA_deg']) and pd.notna(row['RA_hms']):
        ## pad RA_hms with leading zeros
        if len(row['RA_hms']) == 10:
            row['RA_hms'] = '0' + row['RA_hms']
            sestito2024_df.at[idx, 'RA_hms'] = row['RA_hms']
        row['RA_deg'] = coord.ra_hms_to_deg(row['RA_hms'], precision=6)
        sestito2024_df.at[idx, 'RA_deg'] = row['RA_deg']

    if pd.isna(row['DEC_deg']) and pd.notna(row['DEC_dms']):
        row['DEC_deg'] = coord.dec_dms_to_deg(row['DEC_dms'], precision=2)
        sestito2024_df.at[idx, 'DEC_deg'] = row['DEC_deg']

    if pd.isna(row['RA_hms']) and pd.notna(row['RA_deg']):
        row['RA_hms'] = coord.ra_deg_to_hms(float(row['RA_deg']), precision=2)
        sestito2024_df.at[idx, 'RA_hms'] = row['RA_hms']

    if pd.isna(row['DEC_dms']) and pd.notna(row['DEC_deg']):
        row['DEC_dms'] = coord.dec_deg_to_dms(float(row['DEC_deg']), precision=2)
        sestito2024_df.at[idx, 'DEC_dms'] = row['DEC_dms']

## Rename the Abundance columns from ions to elements
for col_original in sestito2024_df.columns:

    if '/H]' in col_original:
        col = re.sub(r'([A-Z][a-z]?)([IV]+)', r'\1 \2', col_original)

        prefix = col.split("[")[0] + "["
        ion = col.split("[")[1].split("/")[0]
        suffix = "/" + col.split("/")[1]

        if len(ion.split(" ")) > 1:
            if ion.split(" ")[1] == "I":

                if get_default_ion(ion.split(" ")[0]) == 2:
                    element = "".join(ion.split(" "))
                elif get_default_ion(ion.split(" ")[0]) == 1:
                    element = ion.split(" ")[0]
                elif get_default_ion(ion.split(" ")[0]) == 0:
                    element = ion.split(" ")[0] #"".join(ion.split(" "))
                else:
                    raise ValueError(f"Unknown ionization state for {ion}")

            elif ion.split(" ")[1] == "II":

                if get_default_ion(ion.split(" ")[0]) == 2:
                    element = ion.split(" ")[0]
                elif get_default_ion(ion.split(" ")[0]) == 1:
                    element = "".join(ion.split(" "))
                elif get_default_ion(ion.split(" ")[0]) == 0:
                    element = ion.split(" ")[0] #"".join(ion.split(" "))
                    print(ion, element)
                else:
                    raise ValueError(f"Unknown ionization state for {ion}")

            else:
                element = ion
        else:
            element = ion
        
        # print(col_original, col, prefix+element+suffix)
        sestito2024_df.rename(columns={col_original: prefix+element+suffix}, inplace=True)

# Categorize columns & reorder dataFrame
columns = list(sestito2024_df.columns)
aux_cols = [
    'Reference','Ref','Name','Simbad_Identifier','Loc','RA_hms','DEC_dms','RA_deg','DEC_deg',
    'Gmag','BP-RP','AV','texp','SN','RV','e_RV','AV.1','Teff','e_Teff','logg','e_logg'
    ]
carbon_cols = [col for col in columns if "[C/" in col]
xh_cols = [col for col in columns if col.startswith("[") and col.endswith("/H]") and col not in carbon_cols]
ul_xh_cols = [col for col in columns if col.startswith("ul[") and col.endswith("/H]") and col not in carbon_cols]
e_xh_cols = [col for col in columns if col.startswith("e_[") and col.endswith("/H]") and col not in carbon_cols]
xfe_cols = [col for col in columns if col.startswith("[") and col.endswith("/Fe]") and col not in carbon_cols]
ul_xfe_cols = [col for col in columns if col.startswith("ul[") and col.endswith("/Fe]") and col not in carbon_cols]
e_xfe_cols = [col for col in columns if col.startswith("e_[") and col.endswith("/Fe]") and col not in carbon_cols]
xy_cols = [col for col in columns if (col.startswith("[") and ("/" in col)) and (col not in xh_cols + xfe_cols + carbon_cols)]
remaining_cols = [col for col in columns if col not in aux_cols + carbon_cols + xh_cols + ul_xh_cols + e_xh_cols + xfe_cols + ul_xfe_cols + e_xfe_cols + xy_cols]

ordered_cols = aux_cols + carbon_cols + xh_cols + ul_xh_cols + e_xh_cols + xfe_cols + ul_xfe_cols + e_xfe_cols + xy_cols + remaining_cols
sestito2024_df = sestito2024_df[ordered_cols]


display(sestito2024_df)

/Users/ayelland/Research/metal-poor-stars/spag/spag/convert.py:890: UserWarning: get_default_ion: K not in defaults, returning 0
  warnings.warn("get_default_ion: {} not in defaults, returning 0".format(elem))
/Users/ayelland/Research/metal-poor-stars/spag/spag/convert.py:890: UserWarning: get_default_ion: Zn not in defaults, returning 0
  warnings.warn("get_default_ion: {} not in defaults, returning 0".format(elem))


,Reference,Ref,Name,Simbad_Identifier,Loc,RA_hms,DEC_dms,RA_deg,DEC_deg,Gmag,BP-RP,AV,texp,SN,RV,e_RV,AV.1,Teff,e_Teff,logg,e_logg,[C/Fe],[C/Fe]c,[C/Fe]f,[Fe/H],[Fe/H],[Fe/H],[Fe/H],[FeII/H],[Na/H],[Mg/H],[Al/H],[Si/H],[K/H],[Ca/H],[Sc/H],[Ti/H],[TiII/H],[Cr/H],[Mn/H],[Co/H],[Ni/H],[Zn/H],[Sr/H],[Y/H],[Ba/H],e_[Fe/H],[Na/Fe],[Mg/Fe],[Al/Fe],[Si/Fe],[K/Fe],[Ca/Fe],[Sc/Fe],[Ti/Fe],[Cr/Fe],[Mn/Fe],[Co/Fe],[Ni/Fe],[Zn/Fe],[Sr/Fe],[Y/Fe],[Ba/Fe],[Eu/Fe],e_[Na/Fe],e_[Mg/Fe],e_[Al/Fe],e_[Si/Fe],e_[K/Fe],e_[Ca/Fe],e_[Sc/Fe],e_[Ti/Fe],e_[Cr/Fe],e_[Mn/Fe],e_[Co/Fe],e_[Ni/Fe],e_[Zn/Fe],e_[Sr/Fe],e_[Y/Fe],e_[Ba/Fe],e_[Eu/Fe]
0,Sestito+2024,SES24,Pristine_184431.86-293145.0,Gaia DR3 4071285193071973504,DW,18:44:31.86,-29:31:44.98,281.13275,-29.52916,15.18,1.54,0.36,6000,20,142.57,0.67,0.3570,4398.51,75.26,0.98,0.11,NaN,NaN,NaN,-2.84,-2.84,-2.84,-2.84,NaN,-2.87,-2.61,NaN,-2.45,-2.26,-2.63,-2.70,-2.84,-2.32,-3.01,NaN,NaN,-2.92,NaN,-2.52,-2.89,-3.24,0.05,-0.03,0.23,NaN,0.38,0.57,0.21,0.13,0.52,-0.17,NaN,NaN,-0.08,NaN,0.32,-0.05,-0.40,NaN,0.08,0.05,NaN,0.07,0.24,0.06,0.07,0.09,0.06,NaN,NaN,0.07,NaN,0.06,0.06,0.08,NaN
1,Sestito+2024,SES24,Pristine_184759.63-315322.5,Gaia DR3 6736966644012267648,DW,18:47:59.63,-31:53:22.49,281.99844,-31.88958,15.41,1.45,0.37,6600,31,184.00,0.41,NaN,4565.00,80.00,1.05,0.11,-0.6,0.77,0.17,-2.30,-2.30,-2.30,-2.30,-2.09,-2.26,-2.07,-2.10,-1.93,-9.99,-2.14,-2.22,-2.30,-1.74,-2.56,-3.29,-2.17,-2.12,-2.17,-2.59,-2.76,-2.54,0.02,0.04,0.24,0.20,0.37,-7.69,0.16,0.08,0.57,-0.26,-0.99,0.13,0.18,0.13,-0.29,-0.46,-0.23,0.0,0.05,0.06,0.05,0.05,0.05,0.04,0.10,0.04,0.08,0.04,0.19,0.05,0.04,0.04,0.06,0.12,0.2
2,Sestito+2024,SES24,Pristine_184843.24-314626.8,Gaia DR3 6736974929031902464,DW,18:48:43.24,-31:46:26.80,282.18017,-31.77411,15.31,1.43,0.42,7200,51,149.02,0.71,0.4202,4597.03,77.66,1.11,0.10,-0.5,0.74,0.24,-3.12,-3.12,-3.12,-3.12,-3.16,-2.88,-2.67,NaN,NaN,-9.99,-3.01,-2.93,NaN,-2.94,-3.34,NaN,NaN,-3.37,NaN,NaN,NaN,-4.16,0.03,0.24,0.45,NaN,NaN,-6.87,0.11,0.20,0.19,-0.22,NaN,NaN,-0.25,NaN,NaN,NaN,-1.03,0.0,0.03,0.04,NaN,NaN,0.06,0.03,0.05,0.10,0.10,NaN,NaN,0.06,NaN,NaN,NaN,0.03,0.2
3,Sestito+2024,SES24,Pristine_184853.44-302718.4,Gaia DR3 6761132195931902208,DW,18:48:53.44,-30:27:18.40,282.22267,-30.45511,15.00,1.60,0.52,5400,23,121.41,0.52,0.5168,4403.00,67.82,0.85,0.10,NaN,NaN,NaN,-2.48,-2.48,-2.48,-2.48,-1.93,-2.66,-2.23,NaN,NaN,-2.03,-2.18,-2.39,-2.69,-1.93,-2.96,-3.23,NaN,-2.63,NaN,NaN,-2.69,-2.86,0.02,-0.18,0.25,NaN,NaN,0.44,0.30,0.09,0.55,-0.49,-0.75,NaN,-0.16,NaN,NaN,-0.22,-0.38,NaN,0.08,0.05,NaN,NaN,0.09,0.04,0.09,0.05,0.06,0.05,NaN,0.03,NaN,NaN,0.05,0.10,NaN
4,Sestito+2024,SES24,Pristine_184957.04-291425.1,Gaia DR3 6761537434686446720,DW,18:49:57.04,-29:14:25.12,282.48767,-29.24031,14.91,1.64,0.49,5400,53,125.57,0.74,0.4873,4323.62,69.64,0.79,0.11,NaN,NaN,NaN,-2.66,-2.66,-2.66,-2.66,-2.38,-2.74,-2.35,-2.38,NaN,-2.27,-2.36,-2.52,-2.70,-1.93,-3.12,-3.61,NaN,-2.47,-2.21,-2.57,-2.58,-2.65,0.02,-0.08,0.31,0.28,NaN,0.39,0.30,0.14,0.73,-0.46,-0.95,NaN,0.19,0.45,0.09,0.08,0.01,-0.2,0.02,0.07,0.05,NaN,0.03,0.05,0.08,0.04,0.11,0.05,NaN,0.09,0.09,0.05,0.07,0.04,0.2
5,Sestito+2024,SES24,Pristine_185053.71-313317.7,Gaia DR3 6736253885603451776,DW,18:50:53.71,-31:33:17.71,282.72379,-31.55492,15.12,1.48,0.49,6000,24,153.21,0.79,0.4877,4564.41,82.48,1.00,0.11,-0.2,0.09,-0.11,-1.22,-1.22,-1.22,-1.22,-1.34,-0.66,-0.85,-0.98,-0.67,-9.99,-0.83,-1.56,-1.29,-1.51,-1.50,-1.46,-1.60,-1.13,-1.33,-1.45,-1.74,-1.58,0.02,0.56,0.37,0.24,0.55,-8.77,0.39,-0.33,-0.28,-0.28,-0.23,-0.38,0.09,-0.10,-0.23,-0.52,-0.35,0.0,0.09,0.15,0.05,0.06,0.05,0.04,0.07,0.04,0.07,0.08,0.23,0.06,0.07,0.06,0.14,0.05,0.2
6,Sestito+2024,SES24,Pristine_185129.00-300942.8,Gaia DR3 6761245956700629504,DW,18:51:29.00,-30:09:42.80,282.87083,-30.16189,15.79,1.37,0.45,9000,56,139.56,0.77,0.4454,4720.59,80.16,1.36,0.11,NaN,NaN,NaN,-3.26,-3.26,-3.26,-3.26,-3.22,-3.22,-2.80,-3.19,-2.51,-2.69,-3.01,-3.07,-3.00,-3.03,-3.69,-4.50,-3.27,-3.43,NaN,-3.63,NaN,-4.41,0.02,0.05,0.47,0.08,

In [5]:
sestito2024_df.to_csv("/Users/ayelland/Research/metal-poor-stars/spag/spag/data/abundance_tables/sestito2024/sestito2024-yelland.csv")
